In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.pmod import Pmod_PWM
from pynq.lib.pmod import Pmod_IO
import time

# Load base overlay
base = BaseOverlay("base.bit")

# Initialize PMODB
pwm_motor_right = Pmod_PWM(base.PMODB, 3)       # Right wheels PWM
pwm_motor_left = Pmod_PWM(base.PMODB, 5)        # Left wheels PWM

pwm_motor_right.stop()
pwm_motor_left.stop()

# Initialize PMODA
dir_rfor = Pmod_IO(base.PMODA, 1, 'out')        # Right wheels reverse
dir_rrev = Pmod_IO(base.PMODA, 2, 'out')        # Right wheels forward

dir_lfor = Pmod_IO(base.PMODA, 5, 'out')        # Left wheels reverse
dir_lrev = Pmod_IO(base.PMODA, 6, 'out')        # Left wheels forward

def control_motor(pwm, direction, speed, forward):
    # Direction
    direction.write(1 if forward else 0)

    # PWM duty cycle
    pwm.generate(1000, speed)  # 1 kHz PWM frequency
    
def defuzz(low, high):
    low_x = 0
    high_x = 0
    
    if low > 0:
        low_x = (low-1)/(-1/110)-100
    else: 
        low_x = 80
    
    if high > 0:
        high_x = (high-1)/(1/110)+100
    else:
        high_x = -80
    
    print([low, high])
    print([low_x, high_x])
    
    speed = ((low_x*low)+(high_x*high))/(low+high)
    if speed == 100:
        speed = speed - 2
    print(speed)
    return speed

# Steering Control System w/ Fuzzy Logic
try:
    # Turn right
    angle = 90
    
    NF = 0
    VC = 0
    PF = 0
    
    # Fuzzification
    # 1) NF
    if angle <= 0:
        if angle <= -180:
            NF = 1 
        else: 
            NF = angle / -180
    
    # 2) VC
    if angle <= 90 and angle >= -90:
        if angle < 0:
            VC = (angle / 90) + 1
        else:
            VC = (angle / -90) + 1
    
    # 3) PF
    if angle >= 0:
        if angle >= 180: 
            PF = 1 
        else:
            PF = angle / 180
    
    print([NF, VC, PF])
    
    # Inference
    L_LOW = NF
    L_HIGH = max(VC, PF)
    
    R_LOW = PF
    R_HIGH = max(NF, VC)
    
    # Defuzzification
    # 1) Left Wheels
    l_speed = defuzz(L_LOW, L_HIGH)
    
    # 2) Right Wheels
    r_speed = defuzz(R_LOW, R_HIGH)
    
    # Motor Control
    if r_speed < 0:
        control_motor(pwm_motor_right, dir_rrev, int(-r_speed), True)
    else:
        control_motor(pwm_motor_right, dir_rfor, int(r_speed), True)
    
    if l_speed < 0:
        control_motor(pwm_motor_left, dir_lrev, int(-l_speed), True)
    else:
        control_motor(pwm_motor_left, dir_lfor, int(l_speed), True)
        
    time.sleep(10)

except KeyboardInterrupt:
    # Stop motors & Clean
    print("Stopping motors...")
    pwm_motor_right.stop()
    pwm_motor_left.stop()